## Importando librerías

In [772]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime
from scipy.stats import truncnorm


## Función 'days_between' para calcular el la diferencia entre 2 fechas, recibe como parámetros:
- d1: fecha mínima
- d2: fecha máxima

retorna:
- un 'int' que se refiere a la diferencia en días entre las dos fechas

In [773]:
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days

## Función 'get_count' para calcular los valores posibles en caso de columnas numéricas, categóricas o dates, recibe como parámetros:
- col: la columna de la cual queremos calcular el total de valores

retorna:
- un 'int' el cual se refiere a la cantidad total de valores posibles

In [774]:
def get_count(col):
    if col['type'] == 'category':
        return len(col['values'])
    elif col['type'] == 'numeric':
        return col['values']['max'] - col['values']['min']
    elif col['type'] == 'date':
        return days_between(col['max'], col['min'])
    else:
        return None

## Función 'getRows' para calcular el número máximo de registros que tendrán el dataframe (Este función y las anteriores son implementadas o usadas solamente en los casos en los que la columna 'random' de cualquier configuración de dataframe sea False), recibe como parámetros:
- conf: la configuración del dataframe del cual queremos obtener la cantidad máxima de registros

retorna:
- un 'int' que corresponde al número mayor entre todos los posibles valores de cada columna

In [775]:
def getRows(conf):
    columns_total_values = [
        get_count(col) for col in conf['columns'] if get_count(col) is not None
    ]

    return max(columns_total_values)

## Función 'generate_unique_id' para crear un str de 16 caracteres de manera aleatoria
retorna:
- un 'str' el cual es un string de 16 caracteres, que pueden ser letras y números

In [776]:
def generate_unique_id():
    caracteres = string.ascii_letters + string.digits
    unique_id = ''.join(random.choices(caracteres, k=16))
    return unique_id

## Función 'generate_unique_ids' para crear 'n' str de 16 caracteres, donde 'n' es el número de registros máximos que tendrá el dataframe dependiendo de la configuración, recibe como parámetros:
- count: el número total de 'ids' que necesitamos

retorna:
- una 'list' que contiene todos los 'ids' distintos

In [777]:
def generate_unique_ids(count):
    unique_ids = set()
    while len(unique_ids) < count:
        unique_ids.add(generate_unique_id())
    return list(unique_ids)


## Función 'generate_truncated_normal_data' para calcular valores de columnas numéricas en caso de que exista media y desviación, recibe como parámetros:
- diccionario que contiene los valores estadísticos
- cantidad de valores que deseamos

retorna:
- un objeto 'list' que contiene todos los valores calculados a partir de la información estadística proporcionada

In [778]:
def generate_truncated_normal_data(min_val, max_val, mean, std, size):
    
    # Calcular los parámetros de la distribución normal truncada
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)

    data = [int(i) for i in data]
    return data

## Función 'generate_col' para calcular cada una de las columnas de cualquier dataframe, esta función recibe como parámetros:
- col: se refiere a la configuración de la columna que deseamos crear.
- conf_list: es la lista de las configuraciones de dataframe, nos servirá para utilizar recursividad, ya que se necesitará para los casos donde existan llave foráneas.
- rows: se refiere al número de valores totales que tendrá nuestra columna.
- df_dict: este es un diccionario de dataframes. En los casos que existan llaves foráneas, nos servirá para verificar que el dataframe al que se hace referencia, ya ha sido creado, en caso contrario, lo crearemos y lo agregaremos a 'df_dict'

retorna: 
- un objeto 'Series' de pandas, el cual es la columna que deseabamos generar

In [779]:
def generate_col(col, conf_list, rows, df_dict):
    if col['type'] == "category":
        return random.choices(col['values'], k=rows)
    elif col['type'] == "numeric":
        if 'mean' in col['values'] and 'std' in col['values']:
            min = col['values']['min']
            max = col['values']['max']
            mean = col['values']['mean']
            std = col['values']['std']
            return generate_truncated_normal_data(min, max, mean, std, rows)
        return [int(random.uniform(col['values']['min'], col['values']['max'])) for i in range(rows)]
    elif col['type'] == "unique":
        return generate_unique_ids(rows)
    elif col['type'] == 'foreign':
        ds_name, col_name_ref = col['values'].split('.')
        if ds_name in df_dict:
            return random.choices(df_dict[ds_name][col_name_ref], k=rows)
        else:
            temp_conf = [c for c in conf_list if c['ds'] == ds_name]
            temp_df = build_dataframe(temp_conf[0], conf_list, df_dict)
            df_dict[ds_name] = temp_df
            return random.choices(df_dict[ds_name][col_name_ref], k=rows)
    else:
        raise ValueError("Tipo de columna no válida")

## Función 'build_dataframe' se utiliza para crear únicamente 1 dataframe, recibe como parámetros:
- conf: la configuración completa del dataframe a crear
- conf_list: la lista de configuraciones, servirá para enviar las configuraciones a 'fun' para los casos recursivos o de llave foránea.
- df_dict: enviamos la diccionarios de diccionarios para ser utilizado en 'fun' 

retorna:
- un objeto 'DataFrame' de pandas

In [780]:
def build_dataframe(conf, conf_list, df_dict):
    rows = conf.get('random_rows') if conf['random'] else getRows(conf)
    df = pd.DataFrame()

    for col in conf['columns']:
        df[col['name']] = generate_col(col, conf_list, rows, df_dict)
    
    return df

## Función 'build_dataframes' sirve para crear la lista funal de dataframes a partir de una lista de configuraciones, recibe solamente un parámetro:
- conf_list: lista de configuraciones de los dataframes a crear.

retorna:
- una 'list' que contiene los diccionarios generados a partir de las configuraciones proporcionadas

In [781]:
def build_dataframes(conf_list):
    df_dict = {}
    result = []

    for conf in conf_list:
        df = build_dataframe(conf, conf_list, df_dict)
        df_dict[conf['ds']] = df
        result.append(df)

    return result

## Ejemplo de uso

In [782]:
d1 = {
        "ds": "dataset1",
        "columns": [
            {
                "name": "segment",
                "type": "category",
                "values": ["Consumer", "Corporate", "Home Office"]
            },
            {
                "name": "id",
                "type": "unique"
            },
            {
                'name': 'test',
                'type': 'numeric',
                'values': {'min': 5, 'max':6}
            }
        ],
        "random": False
    }


d2 = {
        "ds": "dataset2",
        "columns": [
            {
                "name": "id",
                "type": "unique"
            },
            {
                "name": "segment",
                "type": "foreign",
                "values": "dataset1.segment"
            },
            {
                "name": "ship_mode",
                "type": "category",
                "values": ["First Class", "Same Day", "Second Class", "Standard Class"]
            }
        ],
        "random": False
    }


d3 = {
        "ds": "dataset3",
        "columns": [
            {
                "name": "id",
                "type": "unique"
            },
            {
                "name": "segment",
                "type": "foreign",
                "values": "dataset1.segment"
            },
            {
                "name": "ship_mode",
                "type": "foreign",
                "values": "dataset2.ship_mode"
            },
            {
                "name": 'country',
                'type': 'category',
                'values': ['USA', 'HON']
            },
            {
                'name': 'products_cat',
                'type': 'category',
                'values': ['Accesories', 'Art', 'Phones', 'Supplies', 'Chairs', 'Paper']
            },
            {
                'name': 'region',
                'type': 'category',
                'values': ['South', 'North']
            },
            {
                "name": "quantity",
                "type": "numeric",
                "values": {"min": 1, "max": 15}
            },
            {
                "name": "sale price",
                "type": "numeric",
                "values": {"min": 10, "max": 1500, "std": 300,
                "mean": 200}
            }
            ],
        "random": True,
        "random_rows": 1000
    }

configuration_list = [d2, d3, d1]

dataframe_list = build_dataframes(configuration_list)

dataframe_list


[                 id      segment       ship_mode
 0  8HLOzgymTnVYwa6M    Corporate  Standard Class
 1  iWtRHIUgwPTOFr6R  Home Office  Standard Class
 2  Wi6IXJ6W3QpPJC6S    Corporate        Same Day
 3  XjGdMtgxnu4WtlPl  Home Office     First Class,
                    id      segment       ship_mode country products_cat  \
 0    8AEJPHNti6zUPnvZ  Home Office  Standard Class     HON       Phones   
 1    CoIIndigQvtKLIaf    Corporate  Standard Class     USA        Paper   
 2    nQ4NJXUHtIbkyeJQ    Corporate  Standard Class     USA          Art   
 3    k7LwLdchUIGrPLou    Corporate  Standard Class     HON   Accesories   
 4    Tp74bMjnFxKe285E    Corporate  Standard Class     USA       Phones   
 ..                ...          ...             ...     ...          ...   
 995  yXjd1R44mIjnImoh    Corporate        Same Day     HON       Phones   
 996  OxNosmMuoY0pLIZ9    Corporate        Same Day     HON   Accesories   
 997  aoZQT8K5NO7mRNxz    Corporate        Same Day     HON      

# Simulación de Dataset

In [783]:
simulation_extended = dataframe_list[1]

## Análisis Detallado

- Creamos una nueva columna para contabilizar las ocurrencias para cada combinacion distinta
- aggregation: contiene todas las distintas combinaciones entre segment', 'ship_mode', 'country' y 'products_cat', así como el total de ocurrencias que existen en el dataframe original.

He decidido no incluir únicamente la columna categórica 'region' ya que no me aporta información relevante para ser analizada, a diferencia de 'products_cat' que me proporciona más detalle de lo que nos habla el dataframe (que son ventas realizadas).

In [784]:


simulation_extended['n'] = 1

aggregation = (simulation_extended.
               groupby([
                   'segment',
                   'ship_mode',
                   'country',
                #    'region'
                   'products_cat'
               ], as_index = False)
               .agg({
                   'n': ['count']
               })
               )

aggregation

,segment,ship_mode,country,products_cat,n
,,,,,count
0,Corporate,First Class,HON,Accesories,14
1,Corporate,First Class,HON,Art,17
2,Corporate,First Class,HON,Chairs,13
3,Corporate,First Class,HON,Paper,18
4,Corporate,First Class,HON,Phones,9
...,...,...,...,...,...
67,Home Office,Standard Class,USA,Art,18
68,Home Office,Standard Class,USA,Chairs,13
69,Home Office,Standard Class,USA,Paper,16


- Renombramos las columnas de la agregación del paso anterior
- Calculamos las probabilidades de cada combinación, en base a la cantidad total que aparece la combinación en el dataframe original

In [785]:
aggregation.columns = ['segment', 'ship_mode', 'country', 'products_cat', 'count']
aggregation['probs'] = aggregation['count'] / simulation_extended.shape[0]

aggregation

,segment,ship_mode,country,products_cat,count,probs
0,Corporate,First Class,HON,Accesories,14,0.014
1,Corporate,First Class,HON,Art,17,0.017
2,Corporate,First Class,HON,Chairs,13,0.013
3,Corporate,First Class,HON,Paper,18,0.018
4,Corporate,First Class,HON,Phones,9,0.009
...,...,...,...,...,...,...
67,Home Office,Standard Class,USA,Art,18,0.018
68,Home Office,Standard Class,USA,Chairs,13,0.013
69,Home Office,Standard Class,USA,Paper,16,0.016
70,Home Office,Standard Class,USA,Phones,13,0.013


- Generamos una muestra aleatoria a partir de los indices de las combinaciones que están en 'agregation' y utilizando las probabilidades de cada combinación.
- randIndex: contiene los indices generados aleatoriamente.

In [786]:
randIndex = np.random.choice( 
    aggregation.index
    , size=10000 
    , p = list(aggregation['probs'])
)

- Visualizamos las filas del DataFrame 'agregation' cuyos índices están en randIndex, y solo incluye las columnas “segment”, "ship_mode", “country” y “products_cat”. Luego con .loc[2]: Selecciona la fila con el índice 2 de la selección anterior.

In [787]:
OneAreaSubarea = aggregation.loc[randIndex , ['segment', 'ship_mode', 'country', 'products_cat']].loc[2]

segment, ship_m, country, p_cat = OneAreaSubarea.iloc[0]

ship_m

'First Class'

A continuación del dataset original seleccionamos solamente los registros con las combinaciones:
- 'segment': 'Consumer'
- 'ship_mode': 'First Class'
- 'country': 'HON'
- 'products_cat': 'Chairs' 

y podemos ver que tiene 16 registros eso significa que 16/1,000 = 0.016 de representatividad en ese dataset de 1,000 registros

In [788]:
simulation_extended.loc[ 
    ( simulation_extended['segment'] == segment )
    &
    ( simulation_extended['ship_mode'] == ship_m )
    &
    ( simulation_extended['country'] == country )
    &
    ( simulation_extended['products_cat'] == p_cat )
, : ]

,id,segment,ship_mode,country,products_cat,region,quantity,sale price,n
36,vxpgXx1ovm6AfGKC,Corporate,First Class,HON,Chairs,North,6,925,1
86,GFM6mmTgeO6eZU32,Corporate,First Class,HON,Chairs,South,5,588,1
135,QYmBVz1J0R6KM02v,Corporate,First Class,HON,Chairs,North,6,782,1
215,wxAMDB78HwW5CrYE,Corporate,First Class,HON,Chairs,North,13,144,1
345,05ZScKU14c23kFK7,Corporate,First Class,HON,Chairs,South,2,79,1
385,Tjeb2S3ek024nDJX,Corporate,First Class,HON,Chairs,South,6,262,1
388,dD88LdlmCeZHkuT4,Corporate,First Class,HON,Chairs,North,2,446,1
436,pWt8SqRjTSZwyeXA,Corporate,First Class,HON,Chairs,South,5,56,1
594,KJaLoZtFe6Qjnr9W,Corporate,First Class,HON,Chairs,North,9,772,1
754,xMEUFOVI1QWs3VnB,Corporate,First Class,HON,Chairs,North,3,531,1


Ahora, al dividir los registros totales de 'OneAreaSubarea' entre 10,000, observamos que obtenemos un valor muy cercano a la representatividad del dataset original 

In [789]:
OneAreaSubarea.shape[0] / 10000


0.0129

In [790]:
quantityStats = simulation_extended.loc[ 
    ( simulation_extended['segment'] == segment )
    &
    ( simulation_extended['ship_mode'] == ship_m )
    &
    ( simulation_extended['country'] == country )
    &
    ( simulation_extended['products_cat'] == p_cat )
, : ].groupby(
    [ "segment" , "ship_mode", 'country', 'products_cat']
    , as_index = False
).agg(
    {
        "quantity": ["min","max","mean","std"]
    }
)

quantityStats.columns = ['segment', 'ship_mode', 'country', 'products_cat', 'min', 'max', 'mean', 'std']

quantityStats

,segment,ship_mode,country,products_cat,min,max,mean,std
0,Corporate,First Class,HON,Chairs,1,14,6.461538,4.313069


In [828]:
data = generate_truncated_normal_data(
    quantityStats["min"][0]
    , quantityStats["max"][0]
    , quantityStats["mean"][0]
    , quantityStats["std"][0]
    , OneAreaSubarea.shape[0]
)

data

129

In [829]:
OneAreaSubarea['quantity'] = data

OneAreaSubarea

,segment,ship_mode,country,products_cat,quantity
2,Corporate,First Class,HON,Chairs,3
2,Corporate,First Class,HON,Chairs,6
2,Corporate,First Class,HON,Chairs,8
2,Corporate,First Class,HON,Chairs,9
2,Corporate,First Class,HON,Chairs,1
...,...,...,...,...,...
2,Corporate,First Class,HON,Chairs,6
2,Corporate,First Class,HON,Chairs,8
2,Corporate,First Class,HON,Chairs,7
2,Corporate,First Class,HON,Chairs,1


In [793]:
categories = ['segment', 'ship_mode', 'country', 'products_cat']

simulated = aggregation.loc[randIndex, categories]

column_name = 'quantity'

## Creación del dataset simulado

In [836]:
def get_categorical_dataset_simulated(simulation_extended, category_cols, size_):
    # Encontramos las diferentes combinaciones entre las columnas categóricas seleccionadas
    simulation_extended['n'] = 1

    newAgg = simulation_extended.groupby(
        category_cols, 
        as_index = False
    ).agg(
        {
            'n': ['count']
        }
    )

    cols = [c for c in category_cols]
    cols.extend(['count'])
    newAgg.columns = cols

    # Calculamos las probabilidades de cada combinación
    newAgg['prob'] = newAgg['count'] / simulation_extended.shape[0]

    # Creamos una lista de 100,000 elementos, dichos elementos son índices calculados a partir de las diferentes combinaciones y usando sus probabilidades
    randIndex = np.random.choice(
        aggregation.index,
        size=size_,
        p= list(newAgg['prob'])
    )

    # Creamos el dataset simulado seleccionando desde newAgg, los índices que aparecen en randIndex y eligiendo unicamente las columnas categóricas búscadas
    simulated = newAgg.loc[randIndex, category_cols]

    return simulated

In [840]:
category_cols = ['segment', 'ship_mode', 'country', 'products_cat']
size = 100000

final_simulation = get_categorical_dataset_simulated(simulation_extended, cetegory_cols, size)

final_simulation.shape

(100000, 4)

## Generando los valores de las columnas numéricas con la función utilizada en clase

In [841]:
def get_numeric_column_simulated(simulated, df_origin, categories, column_name):
    agg = df_origin.groupby(
        categories
        , as_index = False
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )
    nc = [ c for c in categories ]
    nc.extend( ["min" , "max" , "mean" , "std"] )
    agg.columns = nc

    ColumnSimulated = pd.DataFrame()
    for i in agg.index:
        rs = agg.loc[i]
        OneSegmentShipCountryProd = simulated.loc[i].copy()

        print(f"Valores - min: {rs['min']}, max: {rs['max']}, mean: {rs['mean']}, std: {rs['std']}")
        
        if np.isnan(rs['std']):
            print(f"std: {rs['std']}")
    
        data = generate_truncated_normal_data(
            rs["min"] - 1 if rs["std"] == 0 or np.isnan(rs['std']) else rs["min"]
            , rs["max"] + 1 if rs["std"] == 0 or np.isnan(rs['std']) else rs["max"], 
            rs["mean"]
            , 1 if (rs["std"] <= 0 or np.isnan(rs['std'])) else rs["std"]
            , OneSegmentShipCountryProd.shape[0]
        )
    
        OneSegmentShipCountryProd[column_name] = data
    
        ColumnSimulated = pd.concat( [ ColumnSimulated , OneSegmentShipCountryProd  ] )

    return ColumnSimulated.reset_index(drop=True)

In [843]:
numeric_columns = ["quantity","sale price"]

# final_simulation = simulated.sort_index().reset_index(drop=True).copy()

for nc in numeric_columns:
    ncdf = get_numeric_column_simulated( final_simulation , simulation_extended , cetegory_cols , nc )

    final_simulation = pd.merge( final_simulation , ncdf.loc[ : , [nc] ] , left_index=True , right_index=True )

Valores - min: 1, max: 14, mean: 7.785714285714286, std: 3.886437392316465
Valores - min: 1, max: 14, mean: 7.647058823529412, std: 4.314817152420661
Valores - min: 1, max: 14, mean: 6.461538461538462, std: 4.313068988848208
Valores - min: 1, max: 14, mean: 6.833333333333333, std: 4.328428596805896
Valores - min: 4, max: 14, mean: 8.555555555555555, std: 3.2058973436118903
Valores - min: 2, max: 14, mean: 9.384615384615385, std: 3.9694345008338723
Valores - min: 1, max: 14, mean: 7.571428571428571, std: 4.164211731007364
Valores - min: 1, max: 13, mean: 6.428571428571429, std: 3.776736191970027
Valores - min: 1, max: 14, mean: 7.733333333333333, std: 4.0964560757992245
Valores - min: 1, max: 14, mean: 7.086956521739131, std: 3.462960418725585
Valores - min: 1, max: 13, mean: 7.428571428571429, std: 4.145698154329761
Valores - min: 4, max: 12, mean: 8.857142857142858, std: 3.4364987719368987
Valores - min: 3, max: 14, mean: 9.0, std: 3.927922024247863
Valores - min: 4, max: 14, mean: 9.

In [845]:
final_simulation.to_csv('final_simulation.csv', sep=',')